
# Construção de um Classificador Binário baseado no algoritmo KNN


## Dependências necessárias

In [ ]:
# In[50]:
from __future__ import absolute_import, division, print_function
import pandas as pd
import nltk  
import numpy as np  
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
import heapq
import re  
import io
import math
import csv
# Helper libraries
import matplotlib.pyplot as plt
import seaborn as sns
import functools
import operator
import PIL
import tqdm
import tqdm.auto
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from IPython.display import display
import os

## Carregando variáveis de ambiente

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# ENV_DATASET_FILENAME = os.environ['INPUT_DATASET_FILENAME']
ENV_DATASET_FILENAME = "locations_to_be_labeled.csv"

nameOfTheFile = ENV_DATASET_FILENAME
print("Nome do arquivo que contém o dataset: " + nameOfTheFile)

Nome do arquivo que contém o dataset: locations_to_be_labeled.csv


## Preparação do conjunto de dados

### Carregando o Dataset

In [9]:
# Lendo as features
questions = pd.read_csv("../" + nameOfTheFile, header=0, usecols=[0])
listOfQuestions = []
for row in questions.values:
    listOfQuestions.append(list(row)[0])

print("Quantidade de perguntas: " + str(len(listOfQuestions)) + "\n")
print("Tweet:\n")
for pergunta in listOfQuestions[0:5]:
    print("- " + pergunta)
print("...")
print("\n")

# Lendo as labels
answers = pd.read_csv("../" + nameOfTheFile, header=0, usecols=[1])
listOfAnswers = []
for answer in answers.values:
    listOfAnswers.append(list(answer)[0])
print("Respostas:\n")
for label in listOfAnswers[0:5]:
    print("- " + str(label))
print("...")

Quantidade de perguntas: 1500

Tweet:

- com o cara faixapreta que venceu a dengue ducatambasco haha foi muito bom te rever brother
- dengue me pegou dnv
- prefeitura de santa cruz divulga plano de ação de enfrentamento a dengue chicungunya e zika
- seguimos na luta contra a dengue chikungunya e o zika vírus com essas três doenças não dá para
- g1 pr segunda morte por dengue em paranaguá é confirmada pela saúde  g1 
...


Respostas:

- 1
- 1
- 0
- 0
- 1
...


### Transformando as labels

Essa transformação vai trocar as labels de número para texto, com o objetivo de facilitar a compreensão e leitura de resultados.
0 - generico
1 - doente

In [10]:
label_palavras = []
for item in listOfAnswers:
    if item==0:
        label_palavras.append("generico")
    else:
        label_palavras.append("doente")
print(label_palavras[0:5])
listOfAnswers = label_palavras

['doente', 'doente', 'generico', 'generico', 'doente']


### Remoção de espaçamentos extras, caracteres especiais e pontuações

In [11]:
for i in range(len(listOfQuestions)):
    listOfQuestions [i] = listOfQuestions [i].lower()
    listOfQuestions [i] = re.sub(r'\W',' ',listOfQuestions [i])
    listOfQuestions [i] = re.sub(r'\s+',' ',listOfQuestions [i])

for question in listOfQuestions[0:5]:
    print("- " + question)
print("...")

- com o cara faixapreta que venceu a dengue ducatambasco haha foi muito bom te rever brother
- dengue me pegou dnv
- prefeitura de santa cruz divulga plano de ação de enfrentamento a dengue chicungunya e zika
- seguimos na luta contra a dengue chikungunya e o zika vírus com essas três doenças não dá para
- g1 pr segunda morte por dengue em paranaguá é confirmada pela saúde g1 
...


### Tokenização dos tweets

In [12]:
def tokenize_sentences(listOfSentences):
    listOfSentencesInTokens = []
    for i in range(len(listOfSentences)):
        tokens = nltk.word_tokenize(listOfSentences[i])
        listOfSentencesInTokens.append(tokens)
    return listOfSentencesInTokens


listOfTokenizedQuestions = tokenize_sentences(listOfQuestions)
for tokenizedItem in listOfTokenizedQuestions[0:5]:
      print("- " + str(tokenizedItem))
print("...")

- ['com', 'o', 'cara', 'faixapreta', 'que', 'venceu', 'a', 'dengue', 'ducatambasco', 'haha', 'foi', 'muito', 'bom', 'te', 'rever', 'brother']
- ['dengue', 'me', 'pegou', 'dnv']
- ['prefeitura', 'de', 'santa', 'cruz', 'divulga', 'plano', 'de', 'ação', 'de', 'enfrentamento', 'a', 'dengue', 'chicungunya', 'e', 'zika']
- ['seguimos', 'na', 'luta', 'contra', 'a', 'dengue', 'chikungunya', 'e', 'o', 'zika', 'vírus', 'com', 'essas', 'três', 'doenças', 'não', 'dá', 'para']
- ['g1', 'pr', 'segunda', 'morte', 'por', 'dengue', 'em', 'paranaguá', 'é', 'confirmada', 'pela', 'saúde', 'g1']
...


### Remoção das stop-words

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords[:10]
print("Tamanho das stop-words: " + str(len(stopwords)))

def remove_stop_words(listOfTokenizedSentences):
  sentencesWithNoStopWords = []
  for i in range(len(listOfTokenizedSentences)):
    sentenceWithNoStopWord = []
    for j in range(len(listOfTokenizedSentences[i])):
            if(listOfTokenizedSentences[i][j] not in stopwords):
                sentenceWithNoStopWord.append(listOfTokenizedSentences[i][j])
    sentencesWithNoStopWords.append(sentenceWithNoStopWord)
  return sentencesWithNoStopWords

listOfQuestionsWithNoStopWords = remove_stop_words(listOfTokenizedQuestions)
print("Tamanho: " + str(len(listOfQuestionsWithNoStopWords)))

for itemWithNoStopWord in listOfQuestionsWithNoStopWords[0:5]:
  print("- " + str(itemWithNoStopWord))
print("...")